<h1 align="center">ECE 4438B Advanced Image Processing and Analysis</h1>
<h2 align="center"> Assignment #4</h2>
<h3 align="center"> Yanyu Mu, [ymu2@uwo.ca](mailto:ymu2@uwo.ca?subject=Assignment #4)</h3>
<h4 align="center"> Department of Electrical and Computer Engineering</h4>
<h4 align="center"> Western University</h4>
<h4 align="center"> April 21, 2018</h4>
<h4 align="center"> Submitted to Elvis C.S. Chen</h4>

**Question 1** ($5$ marks)

In [1]:
import numpy as np
import SimpleITK as sitk

In [2]:
def readLandmarksFromFile(fname):
    array = np.loadtxt(fname)
    return array

In [3]:
P = readLandmarksFromFile('P.txt')
V = readLandmarksFromFile('v.txt')
print(P)
print(V)

OSError: P.txt not found.

**Question 2** ($10$ marks)

$
\begin{eqnarray}
U(r) & = & r^2 \log(r^2)\\
r^2 & = & x^2 + y^2
\end{eqnarray}
$

In [ ]:
def computeU (pi,vi):
    size = np.shape(pi)    # Get size of P
    n = size[0]    # Get n -> how many points from P
    r_2 = np.zeros((n,n))    # create r square array
    for i in range (0,n):
        for j in range (0,n):
            r_2[i][j] = (pi[i][0]-pi[j][0])**2 + (pi[i][1]-pi[j][1])**2    # calculate r square for all points in P
            
    U = np.zeros((n,n))    # create U array
    for i in range (0,n):    
        for j in range (0,n):
            if r_2[i][j] == 0:    # if r square = 0 copy to U to avoid log(0) error
                U[i][j] = 0
            else:
                U[i][j] = r_2[i][j]*np.log(r_2[i][j])    
                # calculate All U even though there are only 2 distance it is easier to construct the full array
    return U

In [ ]:
U = computeU(P,V)
print(U)

**Question 3** ($10$ marks)

$
\begin{eqnarray}
K & = &\begin{bmatrix} 0 & U(r_{1,2}) & \cdots & U(r_{1,n}) \\
U(r_{2,1}) & 0 & \cdots & U(r_{2,n}) \\
\cdots & \cdots & \cdots & \cdots \\
U(r_{n,1}) & U(r_{n,2}) & \cdots & 0 \\
\end{bmatrix}_{n \times n}\\
\end{eqnarray}
$

In [ ]:
def genK( P ):
    size = np.shape(P)    # Get size of P
    n = size[0]    # Get n -> how many points from P
    K = np.zeros((n,n))    # create K array
    for i in range (0,n):
        for j in range (0,n):
            if i != j:     # check the zero location
                U = computeU(P,V) 
                K[i][j] = U[i][j]   # fill the rest using U array
            else:
                K[i][j] = 0    # fill zero values
    return K

In [ ]:
K = genK(P)
print(K)

**Question 4** ($10$ marks)

$
L = \left[
\begin{array}{c|c}
K & P\\
\hline
P^{T} & 0
\end{array}
\right]
$

In [ ]:
def genL( P ):
    P_size = np.shape(P)    # Get size of P
    rows = P_size[0]    # Get rows count from P
    cols = P_size[1]    # Get cols count from P
    N_P = np.zeros((rows,cols+1))    # create new P
    for i in range (0,rows):
        for j in range (0,cols+1):
            if j == 0:
                N_P[i][j] = 1    # fill new P first col with 1
            else:
                N_P[i][j] = P[i][j-1]    # copy rest of P
                
    K = genK(P)
    K_size = np.shape(K)    # Get size of K
    L_r = K_size[0] + cols + 1    # determine rows number for L
    L_c = K_size[1] + cols + 1    # determine col number for L
    L = np.zeros((L_r,L_c))    # create empty L array
    for i in range (0,K_size[0]):
        for j in range (0,K_size[1]):
            L[i][j] = K[i][j]    # fill K area
            
    for i in range (K_size[0],L_r):
        for j in range (0,K_size[1]):
            L[i][j] = np.transpose(N_P)[i-K_size[0]][j]    # fill transpose P area

    for i in range (0,K_size[0]):
        for j in range (K_size[1],L_c):
            L[i][j] = N_P[i][j-K_size[1]]    # fill P area
            
    
    return L
    

In [ ]:
L = genL(P)
print(L)

In [ ]:
def solveTPSCoefs(P,V):
    V_size = np.shape(V)    # Get size of V
    rows = V_size[0]    # Get rows count from V
    cols = V_size[1]    # Get cols count from V
    L = genL(P)
    L_shape = np.shape(L)    # Get size of L
    c = L_shape[1]    # Get cols count from L
    V_0 = np.zeros((c,cols))    # create V/0 array
    for i in range (0,rows):
        for j in range (0,cols):
            V_0[i][j] = V[i][j]    # fill V array
    for i in range (rows,c):
        for j in range (0,cols):
            V_0[i][j] = 0    # fill zeros

    b_a = np.matmul(np.linalg.inv(L),V_0)    #  inverse L, and multiple V/0 array
    b_a = np.round((b_a), decimals=3)    # round to 3 decimal
    return b_a

In [ ]:
coef = solveTPSCoefs(P,V)
print(coef)

**Question 5** ($20$ marks)

$
\begin{eqnarray}
x' & = & a_{1,1} + a_{2,1} x + a_{3,1} y + \sum_{j=1}^n w_{j,1} U(|(x_j,y_j,z_j)-(x,y,z)|)\\
y' & = & a_{1,2} + a_{2,2} x + a_{3,2} y + \sum_{j=1}^n w_{j,2} U(|(x_j,y_j,z_j)-(x,y,z)|)\\
\end{eqnarray}
$

$
w(x,y) = \sum_{j} w_j r_j^2 \log (r_j^2)
$



In [5]:
def applyTPSDeformation( P, V, pixelLocation ):
    Coef = solveTPSCoefs(P,V)
    x = pixelLocation[:,0]
    y = pixelLocation[:,1]
    n = Coef.shape[0] - 3    # get the length of w
    U = np.zeros((n,1))    # create U array
    for i in range(0,n):
        r_2[i][j] = (P[i][0]-pixelLocation[i][0])**2 + (P[i][1]-pixelLocation[i][1])**2    # calculate r square for all points in P            
        U[i] = r_2[i][j]*np.log(r_2[i][j])  # calculate U with r square

    xsum = 0
    ysum = 0
    for i in range(0,n):
        xsum += Coef[i][0]*U[i]    # calculate accumulated w*U for x
        ysum += Coef[i][1]*U[i]    # calculate accumulated w*U for y
    n_x = Coef[n][0]+Coef[n+1][0]*x+Coef[n+2][0]*y + xsum    # calculate new x
    n_y = Coef[n][1]+Coef[n+1][1]*x+Coef[n+2][1]*y + ysum    # calculate new y
    n_xy = np.array([n_x,n_y])
 
    
    
    return n_xy

In [6]:
pixelLocation = np.array([[100, 0],[-100, 0],[0, 100],[0, -100]])
new_pixel = applyTPSDeformation(P,V,pixelLocation)
print(new_pixel[:,0])
print(new_pixel[:,1])
print(new_pixel[:,2])
print(new_pixel[:,3])

NameError: name 'P' is not defined

The points have been moved to location shown above. The movement is not exactly the same as P-V because the order of the four points are different.

**Question 6** ($20$ marks)

(**Marks**)
* ($5$ marks) What is the amount of rotations, in degrees, does $U$ and $V^{H}$ represent?
* ($5$ marks) What are the anisotropic scaling factors?
* ($10$ marks) In your own words, describe what the affine transformation $A$ does as part of the TPS transformation. In your discussion, elaborate how landmarks are moved/transformed.

In [ ]:
P2 = readLandmarksFromFile('P2.txt')
V2 = readLandmarksFromFile('V2.txt')
coef2 = solveTPSCoefs(P2,V2)
print(coef2)

In [ ]:
A = np.array([[coef2[6][0], coef2[7][0]], [coef2[6][1], coef2[7][1]]])
U, S, VH = np.linalg.svd(A, full_matrices=True)    # perform singular value decomposition
print('A',A)
print('U:',U)
print('S:',S)
print('VH:',VH)

In [ ]:
np.linalg.det(U)    # check if it's fliped

In [ ]:
np.linalg.det(VH)   # check if it's fliped 

In [ ]:
T = np.array([[-1,0],[0,1]])
U2 = np.matmul(T, U)    # flip U back
VH2 = np.matmul(T, VH)    # flip VH back
print('U:',U2)
print('VH:',VH2)
Urad=np.arctan(U2[0,1]/U2[0,0])    #Find the U rotation in rads 
Udeg=Urad*180/np.pi     #Find U rotation in degrees
print(Udeg)
VHrad=np.arctan(VH2[0,1]/VH2[0,0]) #find the VH rotation in rads 
VHdeg=VHrad*180/np.pi   #Find the VH rotation in degrees 
print(VHdeg)
D2=Udeg-VHdeg    #second rotation
print(D2)

* Degree of U: -53.33; Degree of VH: -44.89 
* Scaling factor: 1.0719, 0.7441
* The affine transformation A is extension by a factor 1.0719 in a direction 44.89 degree clockwise of horizontal, and compression by 0.7441 in the perpendicular direction, followed by a rotation of another 8.45 degree clockwise.